In [17]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from PIL import Image

In [29]:

from keras import optimizers
from keras.layers import Dense, Input, UpSampling2D, Conv2D,MaxPooling2D
from keras.models import Model
import keras

In [25]:
from sklearn.model_selection import train_test_split

# Dataset

In [19]:

(X_train, _), (X_test, _) = keras.datasets.mnist.load_data()



In [20]:
def preprocess(x):
    x = x.astype('float32') / 255.
    return x.reshape(-1, np.prod(x.shape[1:])) # flatten

X_train = preprocess(X_train)
X_test  = preprocess(X_test)

In [26]:
# also create a validation set for training
X_train, X_valid = train_test_split(X_train, test_size=500)

# Model

In [30]:
def make_convolutional_autoencoder():
    # encoding
    inputs = Input(shape=(28, 28, 1))
    x = Conv2D(16, 3, activation='relu', padding='same')(inputs)
    x = MaxPooling2D(padding='same')(x)
    x = Conv2D( 8, 3, activation='relu', padding='same')(x)
    x = MaxPooling2D(padding='same')(x)
    x = Conv2D( 8, 3, activation='relu', padding='same')(x)
    encoded = MaxPooling2D(padding='same')(x)    
    
    # decoding
    x = Conv2D( 8, 3, activation='relu', padding='same')(encoded)
    x = UpSampling2D()(x)
    x = Conv2D( 8, 3, activation='relu', padding='same')(x)
    x = UpSampling2D()(x)
    x = Conv2D(16, 3, activation='relu')(x) # <= padding='valid'!
    x = UpSampling2D()(x)
    decoded = Conv2D(1, 3, activation='sigmoid', padding='same')(x)
    
    # autoencoder
    autoencoder = Model(inputs, decoded)
    autoencoder.compile(optimizer='adam', 
                        loss='binary_crossentropy')
    return autoencoder

# create a convolutional autoencoder
autoencoder = make_convolutional_autoencoder()

# Train 

In [27]:
# reshape the flattened images to 28x28 with 1 channel
X_train = X_train.reshape(-1, 28, 28, 1)
X_valid = X_valid.reshape(-1, 28, 28, 1)
X_test  = X_test.reshape(-1, 28, 28, 1)

In [31]:
def add_noise(x, noise_factor=0.2):
    x = x + np.random.randn(*x.shape) * noise_factor
    x = x.clip(0., 1.)
    return x
    
X_train_noisy = add_noise(X_train)
X_valid_noisy = add_noise(X_valid)
X_test_noisy  = add_noise(X_test)

In [33]:
autoencoder = make_convolutional_autoencoder()
autoencoder.fit(X_train_noisy, X_train, 
                epochs=10, 
                batch_size=128, 
                validation_data=(X_valid_noisy, X_valid))

Train on 59500 samples, validate on 500 samples
Epoch 1/10
59500/59500 [==============================] - 132s 2ms/step - loss: 0.2184 - val_loss: 0.1509
Epoch 2/10
59500/59500 [==============================] - 123s 2ms/step - loss: 0.1421 - val_loss: 0.1327
Epoch 3/10
16512/59500 [=======>......................] - ETA: 1:37 - loss: 0.1330

KeyboardInterrupt: 